In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

In [2]:
s3_client = boto3.client("s3")
sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
key_prefix = "ray_xgboost_distributed"

In [6]:
!wget http://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/05fa7598d4d44836a42fde79b26568b2/v2/airline_14col.data.bz2 airline_14col.data.bz2

--2022-12-08 14:06:24--  http://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/05fa7598d4d44836a42fde79b26568b2/v2/airline_14col.data.bz2
Resolving ee-assets-prod-us-east-1.s3.amazonaws.com (ee-assets-prod-us-east-1.s3.amazonaws.com)... 52.217.170.89, 52.216.41.41, 52.216.209.89, ...
Connecting to ee-assets-prod-us-east-1.s3.amazonaws.com (ee-assets-prod-us-east-1.s3.amazonaws.com)|52.217.170.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1089011924 (1.0G) [binary/octet-stream]
Saving to: ‘airline_14col.data.bz2’

100%[====================================>] 1,089,011,924 52.8MB/s   in 19s    

2022-12-08 14:06:43 (55.7 MB/s) - ‘airline_14col.data.bz2’ saved [1089011924/1089011924]

--2022-12-08 14:06:43--  http://airline_14col.data.bz2/
Resolving airline_14col.data.bz2 (airline_14col.data.bz2)... failed: Name or service not known.
wget: unable to resolve host address ‘airline_14col.data.bz2’
FINISHED --2022-12-08 14:06:43--
Total wall clock time: 19s
Dow

In [7]:
input_data = sess.upload_data("airline_14col.data.bz2", bucket, key_prefix=f"{key_prefix}/input")

In [8]:
subnets=None
security_group_ids=None

In [9]:
# Pytorch Image is used to enable distributed GPU training
estimator_gpu = PyTorch(
    source_dir="src",
    entry_point="train_xgboost_airline.py",
    subnets=subnets,
    security_group_ids=security_group_ids,
    role=role,
    instance_count=2, 
    instance_type="ml.g5.2xlarge",
    framework_version="1.8",
    py_version="py36"
)

In [10]:
estimator_gpu.fit({"train": input_data})

2022-12-08 14:07:40 Starting - Starting the training job...
2022-12-08 14:07:46 Starting - Insufficient capacity error from EC2 while launching instances, retrying!ProfilerReport-1670508460: InProgress
...
2022-12-08 14:08:37 Failed - Training job failed
..

CapacityError: Error for Training job pytorch-training-2022-12-08-14-07-40-126: Failed. Reason: CapacityError: Unable to provision requested ML compute capacity. Please retry using a different ML instance type.